In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from tqdm import tqdm
from glob import glob
from scipy.io import wavfile
import librosa
import glob

In [3]:
import tensorflow as tf
from tensorflow import keras

from keras import models
from keras import layers
from keras import optimizers

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [23]:
def data_loader(file):
    out = []
    for file in tqdm(file):
        #X, data = librosa.load(file)
        #stft = np.abs(librosa.stft(X))
        #X, data = librosa.load(file)
        fs, data = wavfile.read(file)
        out.append(data)    
    out = np.array(out)
    return out

In [24]:
audio_files = ['1.wav', '2.wav']
#audio_files.extend(glob.glob('train/*.wav'))

In [25]:
# Wav 파일로부터 Feature를 만듭니다.
x_data = data_loader(audio_files)
x_data = x_data[:, ::8] # 매 8번째 데이터만 사용
# x_data = x_data / 30000 # 최대값 30,000 을 나누어 데이터 정규화
x_data.shape

100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2004.45it/s]


(2, 2000)

In [21]:
x_data = x_data.reshape(x_data.shape[0], x_data.shape[1], 1)

IndexError: tuple index out of range

In [ ]:
# Wav 파일로부터 Feature를 만듭니다.
x_data = data_loader(audio_files)
# x_data = x_data[:, ::8] # 매 8번째 데이터만 사용
# x_data = x_data / 30000 # 최대값 30,000 을 나누어 데이터 정규화
x_data = x_data.reshape(x_data.shape[0], x_data.shape[1], 1) # CNN 모델에 넣기 위한 데이터 shape 변경

# 정답 값을 불러옵니다
y_data = pd.read_csv('train_answer.csv', index_col=0)
y_data = y_data.values

# Feature, Label Shape을 확인합니다.
x_data.shape, y_data.shape

  4%|██▋                                                                         | 3598/100000 [02:01<48:02, 33.44it/s]

In [3]:
x_data = x_data.reshape(x_data.shape[0], x_data.shape[1], 1) # CNN 모델에 넣기 위한 데이터 shape 변경

# 정답 값을 불러옵니다
y_data = pd.read_csv('train_answer.csv', index_col=0)
y_data = y_data.values

# Feature, Label Shape을 확인합니다.
x_data.shape, y_data.shape

((100000, 193, 1), (100000, 30))

In [ ]:
model = Sequential()
for i in range(2):
    model.add(LSTM(32, input_shape=(x_data.shape[1], 1), stateful=False, return_sequences=True))
    model.add(Dropout(0.3))
model.add(LSTM(32, input_shape=(x_data.shape[1], 1), stateful=False))
model.add(Dense(30)) # output = 1
model.compile(loss=tf.keras.losses.KLDivergence(), optimizer='adam')

model.summary()

In [ ]:
# 모델 폴더를 생성합니다.
model_path = 'model/'
if not os.path.exists(model_path):
  os.mkdir(model_path)

# Validation 점수가 가장 좋은 모델만 저장합니다.
model_file_path = model_path + 'Epoch_{epoch:03d}_Val_{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath=model_file_path, monitor='val_loss', verbose=1, save_best_only=True)

# 10회 간 Validation 점수가 좋아지지 않으면 중지합니다.
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# 모델을 학습시킵니다.
history = model.fit(
    x_data, y_data, 
    epochs=100, batch_size=256, validation_split=0.2, shuffle=True,
    callbacks=[checkpoint, early_stopping]
)

In [3]:
print('Pandas: %s'%(pd.__version__))
print('Numpy: %s'%(np.__version__))
print('Scipy: %s'%(scipy.__version__))
print('Tensorflow: %s'%(tf.__version__))
print('Keras: %s'%(keras.__version__))

Pandas: 0.25.1
Numpy: 1.16.5
Scipy: 1.4.1
Tensorflow: 2.1.0
Keras: 2.2.4-tf


In [7]:
# wav 파일로부터 데이터를 불러오는 함수, 파일 경로를 리스트 형태로 입력
def data_loader(files):
    out = []
    for file in tqdm(files):
        fs, data = wavfile.read(file)
        out.append(data)    
    out = np.array(out)
    return out

In [8]:
# Wav 파일로부터 Feature를 만듭니다.
x_data = glob('train/*.wav')
x_data = data_loader(x_data)
x_data = x_data[:, ::8] # 매 8번째 데이터만 사용
x_data = x_data / 30000 # 최대값 30,000 을 나누어 데이터 정규화
x_data = x_data.reshape(x_data.shape[0], x_data.shape[1], 1) # CNN 모델에 넣기 위한 데이터 shape 변경

# 정답 값을 불러옵니다
y_data = pd.read_csv('train_answer.csv', index_col=0)
y_data = y_data.values

# Feature, Label Shape을 확인합니다.
x_data.shape, y_data.shape

TypeError: 'module' object is not callable

In [ ]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping

K.clear_session()
model = Sequential() # Sequeatial Model
model.add(LSTM(20, input_shape=(2000, 1))) # (timestep, feature)
model.add(Dense(30)) # output = 1
model.compile(loss=tf.keras.losses.KLDivergence(), optimizer='adam')

model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

model.fit(x_data, y_data, epochs=100,
          batch_size=30, verbose=1, callbacks=[early_stop])

In [ ]:
# 검증 wav 파일로부터 Feature를 만듭니다.
x_test = glob('test/*.wav')
x_test = data_loader(x_test)
x_test = x_test / 30000
x_test = x_test[:, ::8]
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [ ]:
y_pred = model.predict(x_test)
# 예측 결과로 제출 파일을 생성합니다.
submission = pd.read_csv('submission.csv', index_col=0)
submission.loc[:, :] = y_pred
submission.to_csv('submission.csv')

In [ ]:
###########################################

In [5]:
def feature(fn):
    X, sample_rate = librosa.load(fn)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

In [6]:
def audio_to_signal(filenames):
    rows = len(filenames)
    features= np.zeros((rows,193))
    i = 0
    for fn in filenames:
        try:
            mfccs, chroma, mel, contrast,tonnetz = feature(fn)
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
        except:
            print("error : "+fn)
        else:
            features[i] = ext_features
            i += 1
            print("succes : "+fn)
    return features

In [7]:
audio_files = []
audio_files.extend(glob.glob('train/*.wav'))

In [8]:
print(len(audio_files))

100000


In [ ]:
files = audio_files
x_data = audio_to_signal(files)

succes : train\train_00000.wav
succes : train\train_00001.wav
succes : train\train_00002.wav
succes : train\train_00003.wav
succes : train\train_00004.wav
succes : train\train_00005.wav
succes : train\train_00006.wav
succes : train\train_00007.wav
succes : train\train_00008.wav
succes : train\train_00009.wav
succes : train\train_00010.wav
succes : train\train_00011.wav
succes : train\train_00012.wav
succes : train\train_00013.wav
succes : train\train_00014.wav
succes : train\train_00015.wav
succes : train\train_00016.wav
succes : train\train_00017.wav
succes : train\train_00018.wav
succes : train\train_00019.wav
succes : train\train_00020.wav
succes : train\train_00021.wav
succes : train\train_00022.wav
succes : train\train_00023.wav
succes : train\train_00024.wav
succes : train\train_00025.wav
succes : train\train_00026.wav
succes : train\train_00027.wav
succes : train\train_00028.wav
succes : train\train_00029.wav
succes : train\train_00030.wav
succes : train\train_00031.wav
succes :

succes : train\train_00528.wav
succes : train\train_00529.wav
succes : train\train_00530.wav
succes : train\train_00531.wav
succes : train\train_00532.wav
succes : train\train_00533.wav
succes : train\train_00534.wav
succes : train\train_00535.wav
succes : train\train_00536.wav
succes : train\train_00537.wav
succes : train\train_00538.wav
succes : train\train_00539.wav
succes : train\train_00540.wav
succes : train\train_00541.wav
succes : train\train_00542.wav
succes : train\train_00543.wav
succes : train\train_00544.wav
succes : train\train_00545.wav
succes : train\train_00546.wav
succes : train\train_00547.wav
succes : train\train_00548.wav
succes : train\train_00549.wav
succes : train\train_00550.wav
succes : train\train_00551.wav
succes : train\train_00552.wav
succes : train\train_00553.wav
succes : train\train_00554.wav
succes : train\train_00555.wav
succes : train\train_00556.wav
succes : train\train_00557.wav
succes : train\train_00558.wav
succes : train\train_00559.wav
succes :

succes : train\train_01056.wav
succes : train\train_01057.wav
succes : train\train_01058.wav
succes : train\train_01059.wav
succes : train\train_01060.wav
succes : train\train_01061.wav
succes : train\train_01062.wav
succes : train\train_01063.wav
succes : train\train_01064.wav
succes : train\train_01065.wav
succes : train\train_01066.wav
succes : train\train_01067.wav
succes : train\train_01068.wav
succes : train\train_01069.wav
succes : train\train_01070.wav
succes : train\train_01071.wav
succes : train\train_01072.wav
succes : train\train_01073.wav
succes : train\train_01074.wav
succes : train\train_01075.wav
succes : train\train_01076.wav
succes : train\train_01077.wav
succes : train\train_01078.wav
succes : train\train_01079.wav
succes : train\train_01080.wav
succes : train\train_01081.wav
succes : train\train_01082.wav
succes : train\train_01083.wav
succes : train\train_01084.wav
succes : train\train_01085.wav
succes : train\train_01086.wav
succes : train\train_01087.wav
succes :

succes : train\train_01584.wav
succes : train\train_01585.wav
succes : train\train_01586.wav
succes : train\train_01587.wav
succes : train\train_01588.wav
succes : train\train_01589.wav
succes : train\train_01590.wav
succes : train\train_01591.wav
succes : train\train_01592.wav
succes : train\train_01593.wav
succes : train\train_01594.wav
succes : train\train_01595.wav
succes : train\train_01596.wav
succes : train\train_01597.wav
succes : train\train_01598.wav
succes : train\train_01599.wav
succes : train\train_01600.wav
succes : train\train_01601.wav
succes : train\train_01602.wav
succes : train\train_01603.wav
succes : train\train_01604.wav
succes : train\train_01605.wav
succes : train\train_01606.wav
succes : train\train_01607.wav
succes : train\train_01608.wav
succes : train\train_01609.wav
succes : train\train_01610.wav
succes : train\train_01611.wav
succes : train\train_01612.wav
succes : train\train_01613.wav
succes : train\train_01614.wav
succes : train\train_01615.wav
succes :

succes : train\train_02112.wav
succes : train\train_02113.wav
succes : train\train_02114.wav
succes : train\train_02115.wav
succes : train\train_02116.wav
succes : train\train_02117.wav
succes : train\train_02118.wav
succes : train\train_02119.wav
succes : train\train_02120.wav
succes : train\train_02121.wav
succes : train\train_02122.wav
succes : train\train_02123.wav
succes : train\train_02124.wav
succes : train\train_02125.wav
succes : train\train_02126.wav
succes : train\train_02127.wav
succes : train\train_02128.wav
succes : train\train_02129.wav
succes : train\train_02130.wav
succes : train\train_02131.wav
succes : train\train_02132.wav
succes : train\train_02133.wav
succes : train\train_02134.wav
succes : train\train_02135.wav
succes : train\train_02136.wav
succes : train\train_02137.wav
succes : train\train_02138.wav
succes : train\train_02139.wav
succes : train\train_02140.wav
succes : train\train_02141.wav
succes : train\train_02142.wav
succes : train\train_02143.wav
succes :

In [ ]:
x_data.shape

In [ ]:
x_data = x_data.reshape(x_data.shape[0], x_data.shape[1], 1) # CNN 모델에 넣기 위한 데이터 shape 변경

# 정답 값을 불러옵니다
y_data = pd.read_csv('train_answer.csv', index_col=0)
y_data = y_data.values

# Feature, Label Shape을 확인합니다.
x_data.shape, y_data.shape

In [ ]:
# 모델을 만듭니다.
model = Sequential()
model.add(Conv1D(16, 32, activation='relu', input_shape=(x_data.shape[1], x_data.shape[2])))
model.add(MaxPooling1D())
model.add(Conv1D(16, 32, activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(16, 32, activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(16, 32, activation='relu'))
model.add(MaxPooling1D())
model.add(Conv1D(16, 32, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(30, activation='softmax'))
model.compile(loss=tf.keras.losses.KLDivergence(), optimizer='adam')

In [ ]:
# 모델 폴더를 생성합니다.
model_path = 'model/'
if not os.path.exists(model_path):
  os.mkdir(model_path)

# Validation 점수가 가장 좋은 모델만 저장합니다.
model_file_path = model_path + 'Epoch_{epoch:03d}_Val_{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath=model_file_path, monitor='val_loss', verbose=1, save_best_only=True)

# 10회 간 Validation 점수가 좋아지지 않으면 중지합니다.
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# 모델을 학습시킵니다.
history = model.fit(
    x_data, y_data, 
    epochs=100, batch_size=256, validation_split=0.2, shuffle=True,
    callbacks=[checkpoint, early_stopping]
)

In [ ]:
# 훈련 결과를 확인합니다.
plt.plot(history.epoch, history.history['loss'], '-o', label='training_loss')
plt.plot(history.epoch, history.history['val_loss'], '-o', label='validation_loss')
plt.legend()
plt.xlim(left=0)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

In [ ]:
# 검증 wav 파일로부터 Feature를 만듭니다.
x_test = glob('test/*.wav')

x_test = audio_to_signal(x_data)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [ ]:
# 가장 좋은 모델의 weight를 불러옵니다.
weigth_file = glob('model/*.hdf5')[-1]
print(weigth_file)
model.load_weights(weigth_file)

# 예측 수행
y_pred = model.predict(x_test)

# 예측 결과로 제출 파일을 생성합니다.
submission = pd.read_csv('submission.csv', index_col=0)
submission.loc[:, :] = y_pred
submission.to_csv('submission.csv')

In [ ]:
############################################################################

In [53]:
def data_loader(files):
    out = []
    data = np.zeros((rows,193))
    for file in tqdm(files):
        X, sample_rate = librosa.load(files)
        stft = np.abs(librosa.stft(X))
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    
        data = np.hstack([mfccs,mel,contrast,tonnetz])
        out.append(data)    
    out = np.array(out)
    return out

In [39]:
import librosa
def feature(fn):
    X, sample_rate = librosa.load(fn)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

In [54]:
x_data = glob('train/*.wav')
x_data = data_loader(x_data)

NameError: name 'rows' is not defined

In [8]:
# Wav 파일로부터 Feature를 만듭니다.
x_data = glob('train/*.wav')
x_data = data_loader(x_data)
x_data = x_data[:, ::8] # 매 8번째 데이터만 사용
x_data = x_data / 30000 # 최대값 30,000 을 나누어 데이터 정규화


 14%|██████████▏                                                               | 13750/100000 [00:23<10:13, 140.65it/s]


 16%|███████████▋                                                              | 15712/100000 [00:38<11:07, 126.33it/s]


 18%|█████████████                                                             | 17656/100000 [00:53<09:39, 142.06it/s]


 20%|██████████████▋                                                           | 19796/100000 [01:07<10:07, 132.00it/s]


 22%|████████████████▏                                                         | 21810/100000 [01:22<09:53, 131.85it/s]


...

 92%|████████████████████████████████████████████████████████████████████▍     | 92500/100000 [10:35<00:55, 134.55it/s]


 94%|█████████████████████████████████████████████████████████████████████▉    | 94434/100000 [10:50<00:39, 142.10it/s]


 96%|███████████████████████████████████████████████████████████████████████▎  | 96399/100000 [11:06<00:28, 126.85it/s]


 98%|████████████████████████████████████████████████████████████████████████▊ | 98337/100000 [11:20<00:15, 109.35it/s]


100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [11:33<00:00, 144.19it/s]


In [33]:
x_data = x_data.reshape(x_data.shape[0], x_data.shape[1], 1) # CNN 모델에 넣기 위한 데이터 shape 변경

# 정답 값을 불러옵니다
y_data = pd.read_csv('train_answer.csv', index_col=0)
y_data = y_data.values

# Feature, Label Shape을 확인합니다.
x_data.shape, y_data.shape

((100000, 2000, 1), (100000, 30))

In [16]:
look_back = 2000

In [35]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping

K.clear_session()
model = Sequential() # Sequeatial Model
model.add(LSTM(20, input_shape=(2000, 1))) # (timestep, feature)
model.add(Dense(30)) # output = 1
model.compile(loss=tf.keras.losses.KLDivergence(), optimizer='adam')

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                1760      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                630       
Total params: 2,390
Trainable params: 2,390
Non-trainable params: 0
_________________________________________________________________


In [18]:
model = Sequential()

model.add(LSTM(32, batch_input_shape=(1, look_back, 1), stateful= False, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32, batch_input_shape=(1, look_back, 1), stateful=True, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32, batch_input_shape=(1, look_back, 1), stateful=True))
model.add(Dropout(0.3))
model.add(Dense(30))
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (1, 2000, 32)             4352      
_________________________________________________________________
dropout_10 (Dropout)         (1, 2000, 32)             0         
_________________________________________________________________
lstm_11 (LSTM)               (1, 2000, 32)             8320      
_________________________________________________________________
dropout_11 (Dropout)         (1, 2000, 32)             0         
_________________________________________________________________
lstm_12 (LSTM)               (1, 32)                   8320      
_________________________________________________________________
dropout_12 (Dropout)         (1, 32)                   0         
_________________________________________________________________
dense_4 (Dense)              (1, 30)                  

In [13]:
class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

In [36]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

model.fit(x_data, y_data, epochs=100,
          batch_size=30, verbose=1, callbacks=[early_stop])

Epoch 1/100


  9390/100000 [=>............................] - ETA: 59:49 - loss: 0.00 - ETA: 49:40 - loss: 0.00 - ETA: 46:06 - loss: 0.00 - ETA: 44:08 - loss: 0.00 - ETA: 43:11 - loss: 0.00 - ETA: 42:25 - loss: 0.00 - ETA: 41:24 - loss: 0.00 - ETA: 40:51 - loss: 0.00 - ETA: 41:03 - loss: 0.00 - ETA: 40:58 - loss: 0.00 - ETA: 40:40 - loss: 0.00 - ETA: 40:24 - loss: 0.00 - ETA: 40:10 - loss: 0.00 - ETA: 40:06 - loss: 0.00 - ETA: 39:56 - loss: 0.00 - ETA: 39:47 - loss: 0.00 - ETA: 39:34 - loss: 0.00 - ETA: 39:20 - loss: 0.00 - ETA: 39:17 - loss: 0.00 - ETA: 39:02 - loss: 0.00 - ETA: 38:52 - loss: 0.00 - ETA: 38:46 - loss: 0.00 - ETA: 38:33 - loss: 0.00 - ETA: 38:26 - loss: 0.00 - ETA: 38:15 - loss: 0.00 - ETA: 38:06 - loss: 0.00 - ETA: 37:57 - loss: 0.00 - ETA: 37:49 - loss: 0.00 - ETA: 37:48 - loss: 0.00 - ETA: 37:39 - loss: 0.00 - ETA: 37:31 - loss: 0.00 - ETA: 37:24 - loss: 0.00 - ETA: 37:17 - loss: 0.00 - ETA: 37:13 - loss: 0.00 - ETA: 37:08 - loss: 0.00 - ETA: 37:06 - loss: 0.00 - ETA: 37:00 - lo

 75330/100000 [=====================>........] - ETA: 14:36 - loss: 0.00 - ETA: 14:35 - loss: 0.00 - ETA: 14:35 - loss: 0.00 - ETA: 14:34 - loss: 0.00 - ETA: 14:33 - loss: 0.00 - ETA: 14:32 - loss: 0.00 - ETA: 14:32 - loss: 0.00 - ETA: 14:31 - loss: 0.00 - ETA: 14:30 - loss: 0.00 - ETA: 14:29 - loss: 0.00 - ETA: 14:29 - loss: 0.00 - ETA: 14:28 - loss: 0.00 - ETA: 14:27 - loss: 0.00 - ETA: 14:26 - loss: 0.00 - ETA: 14:26 - loss: 0.00 - ETA: 14:25 - loss: 0.00 - ETA: 14:24 - loss: 0.00 - ETA: 14:23 - loss: 0.00 - ETA: 14:23 - loss: 0.00 - ETA: 14:22 - loss: 0.00 - ETA: 14:21 - loss: 0.00 - ETA: 14:20 - loss: 0.00 - ETA: 14:20 - loss: 0.00 - ETA: 14:19 - loss: 0.00 - ETA: 14:18 - loss: 0.00 - ETA: 14:17 - loss: 0.00 - ETA: 14:17 - loss: 0.00 - ETA: 14:16 - loss: 0.00 - ETA: 14:15 - loss: 0.00 - ETA: 14:14 - loss: 0.00 - ETA: 14:14 - loss: 0.00 - ETA: 14:13 - loss: 0.00 - ETA: 14:12 - loss: 0.00 - ETA: 14:11 - loss: 0.00 - ETA: 14:11 - loss: 0.00 - ETA: 14:10 - loss: 0.00 - ETA: 14:09 - lo

 84750/100000 [========================>.....] - ETA: 10:36 - loss: 0.00 - ETA: 10:35 - loss: 0.00 - ETA: 10:34 - loss: 0.00 - ETA: 10:34 - loss: 0.00 - ETA: 10:33 - loss: 0.00 - ETA: 10:32 - loss: 0.00 - ETA: 10:31 - loss: 0.00 - ETA: 10:31 - loss: 0.00 - ETA: 10:30 - loss: 0.00 - ETA: 10:29 - loss: 0.00 - ETA: 10:28 - loss: 0.00 - ETA: 10:28 - loss: 0.00 - ETA: 10:27 - loss: 0.00 - ETA: 10:26 - loss: 0.00 - ETA: 10:25 - loss: 0.00 - ETA: 10:24 - loss: 0.00 - ETA: 10:24 - loss: 0.00 - ETA: 10:23 - loss: 0.00 - ETA: 10:22 - loss: 0.00 - ETA: 10:21 - loss: 0.00 - ETA: 10:21 - loss: 0.00 - ETA: 10:20 - loss: 0.00 - ETA: 10:19 - loss: 0.00 - ETA: 10:18 - loss: 0.00 - ETA: 10:18 - loss: 0.00 - ETA: 10:17 - loss: 0.00 - ETA: 10:16 - loss: 0.00 - ETA: 10:15 - loss: 0.00 - ETA: 10:14 - loss: 0.00 - ETA: 10:14 - loss: 0.00 - ETA: 10:13 - loss: 0.00 - ETA: 10:12 - loss: 0.00 - ETA: 10:11 - loss: 0.00 - ETA: 10:11 - loss: 0.00 - ETA: 10:10 - loss: 0.00 - ETA: 10:09 - loss: 0.00 - ETA: 10:08 - lo

 94170/100000 [===========================>..] - ETA: 6:34 - loss: 0.008 - ETA: 6:33 - loss: 0.008 - ETA: 6:32 - loss: 0.008 - ETA: 6:31 - loss: 0.008 - ETA: 6:31 - loss: 0.008 - ETA: 6:30 - loss: 0.008 - ETA: 6:29 - loss: 0.008 - ETA: 6:28 - loss: 0.008 - ETA: 6:27 - loss: 0.008 - ETA: 6:27 - loss: 0.008 - ETA: 6:26 - loss: 0.008 - ETA: 6:25 - loss: 0.008 - ETA: 6:24 - loss: 0.008 - ETA: 6:24 - loss: 0.008 - ETA: 6:23 - loss: 0.008 - ETA: 6:22 - loss: 0.008 - ETA: 6:21 - loss: 0.008 - ETA: 6:20 - loss: 0.008 - ETA: 6:20 - loss: 0.008 - ETA: 6:19 - loss: 0.008 - ETA: 6:18 - loss: 0.008 - ETA: 6:17 - loss: 0.008 - ETA: 6:17 - loss: 0.008 - ETA: 6:16 - loss: 0.008 - ETA: 6:15 - loss: 0.008 - ETA: 6:14 - loss: 0.008 - ETA: 6:14 - loss: 0.008 - ETA: 6:13 - loss: 0.008 - ETA: 6:12 - loss: 0.008 - ETA: 6:11 - loss: 0.008 - ETA: 6:10 - loss: 0.008 - ETA: 6:10 - loss: 0.008 - ETA: 6:09 - loss: 0.008 - ETA: 6:08 - loss: 0.008 - ETA: 6:07 - loss: 0.008 - ETA: 6:07 - loss: 0.008 - ETA: 6:06 - los

...

  9420/100000 [=>............................] - ETA: 44:28 - loss: 0.00 - ETA: 43:29 - loss: 0.00 - ETA: 43:21 - loss: 0.00 - ETA: 43:34 - loss: 0.00 - ETA: 43:41 - loss: 0.00 - ETA: 43:33 - loss: 0.00 - ETA: 43:22 - loss: 0.00 - ETA: 43:19 - loss: 0.00 - ETA: 43:11 - loss: 0.00 - ETA: 43:08 - loss: 0.00 - ETA: 43:11 - loss: 0.00 - ETA: 43:12 - loss: 0.00 - ETA: 43:08 - loss: 0.00 - ETA: 43:10 - loss: 0.00 - ETA: 43:07 - loss: 0.00 - ETA: 43:10 - loss: 0.00 - ETA: 43:09 - loss: 0.00 - ETA: 43:13 - loss: 0.00 - ETA: 43:11 - loss: 0.00 - ETA: 43:14 - loss: 0.00 - ETA: 43:11 - loss: 0.00 - ETA: 43:06 - loss: 0.00 - ETA: 43:06 - loss: 0.00 - ETA: 43:12 - loss: 0.00 - ETA: 43:10 - loss: 0.00 - ETA: 43:07 - loss: 0.00 - ETA: 43:05 - loss: 0.00 - ETA: 43:05 - loss: 0.00 - ETA: 43:04 - loss: 0.00 - ETA: 43:03 - loss: 0.00 - ETA: 43:03 - loss: 0.00 - ETA: 43:03 - loss: 0.00 - ETA: 43:02 - loss: 0.00 - ETA: 43:00 - loss: 0.00 - ETA: 43:00 - loss: 0.00 - ETA: 42:57 - loss: 0.00 - ETA: 42:59 - lo

 75360/100000 [=====================>........] - ETA: 14:56 - loss: 0.00 - ETA: 14:55 - loss: 0.00 - ETA: 14:54 - loss: 0.00 - ETA: 14:53 - loss: 0.00 - ETA: 14:53 - loss: 0.00 - ETA: 14:52 - loss: 0.00 - ETA: 14:51 - loss: 0.00 - ETA: 14:50 - loss: 0.00 - ETA: 14:49 - loss: 0.00 - ETA: 14:49 - loss: 0.00 - ETA: 14:48 - loss: 0.00 - ETA: 14:47 - loss: 0.00 - ETA: 14:46 - loss: 0.00 - ETA: 14:45 - loss: 0.00 - ETA: 14:45 - loss: 0.00 - ETA: 14:44 - loss: 0.00 - ETA: 14:43 - loss: 0.00 - ETA: 14:42 - loss: 0.00 - ETA: 14:41 - loss: 0.00 - ETA: 14:41 - loss: 0.00 - ETA: 14:40 - loss: 0.00 - ETA: 14:39 - loss: 0.00 - ETA: 14:38 - loss: 0.00 - ETA: 14:38 - loss: 0.00 - ETA: 14:37 - loss: 0.00 - ETA: 14:36 - loss: 0.00 - ETA: 14:35 - loss: 0.00 - ETA: 14:34 - loss: 0.00 - ETA: 14:34 - loss: 0.00 - ETA: 14:33 - loss: 0.00 - ETA: 14:32 - loss: 0.00 - ETA: 14:31 - loss: 0.00 - ETA: 14:30 - loss: 0.00 - ETA: 14:30 - loss: 0.00 - ETA: 14:29 - loss: 0.00 - ETA: 14:28 - loss: 0.00 - ETA: 14:27 - lo

 84780/100000 [========================>.....] - ETA: 10:48 - loss: 0.00 - ETA: 10:47 - loss: 0.00 - ETA: 10:46 - loss: 0.00 - ETA: 10:45 - loss: 0.00 - ETA: 10:44 - loss: 0.00 - ETA: 10:44 - loss: 0.00 - ETA: 10:43 - loss: 0.00 - ETA: 10:42 - loss: 0.00 - ETA: 10:41 - loss: 0.00 - ETA: 10:40 - loss: 0.00 - ETA: 10:40 - loss: 0.00 - ETA: 10:39 - loss: 0.00 - ETA: 10:38 - loss: 0.00 - ETA: 10:37 - loss: 0.00 - ETA: 10:37 - loss: 0.00 - ETA: 10:36 - loss: 0.00 - ETA: 10:35 - loss: 0.00 - ETA: 10:34 - loss: 0.00 - ETA: 10:33 - loss: 0.00 - ETA: 10:33 - loss: 0.00 - ETA: 10:32 - loss: 0.00 - ETA: 10:31 - loss: 0.00 - ETA: 10:30 - loss: 0.00 - ETA: 10:29 - loss: 0.00 - ETA: 10:29 - loss: 0.00 - ETA: 10:28 - loss: 0.00 - ETA: 10:27 - loss: 0.00 - ETA: 10:26 - loss: 0.00 - ETA: 10:25 - loss: 0.00 - ETA: 10:25 - loss: 0.00 - ETA: 10:24 - loss: 0.00 - ETA: 10:23 - loss: 0.00 - ETA: 10:22 - loss: 0.00 - ETA: 10:21 - loss: 0.00 - ETA: 10:21 - loss: 0.00 - ETA: 10:20 - loss: 0.00 - ETA: 10:19 - lo

 94200/100000 [===========================>..] - ETA: 6:39 - loss: 0.008 - ETA: 6:39 - loss: 0.008 - ETA: 6:38 - loss: 0.008 - ETA: 6:37 - loss: 0.008 - ETA: 6:36 - loss: 0.008 - ETA: 6:35 - loss: 0.008 - ETA: 6:35 - loss: 0.008 - ETA: 6:34 - loss: 0.008 - ETA: 6:33 - loss: 0.008 - ETA: 6:32 - loss: 0.008 - ETA: 6:32 - loss: 0.008 - ETA: 6:31 - loss: 0.008 - ETA: 6:30 - loss: 0.008 - ETA: 6:29 - loss: 0.008 - ETA: 6:28 - loss: 0.008 - ETA: 6:28 - loss: 0.008 - ETA: 6:27 - loss: 0.008 - ETA: 6:26 - loss: 0.008 - ETA: 6:25 - loss: 0.008 - ETA: 6:24 - loss: 0.008 - ETA: 6:24 - loss: 0.008 - ETA: 6:23 - loss: 0.008 - ETA: 6:22 - loss: 0.008 - ETA: 6:21 - loss: 0.008 - ETA: 6:20 - loss: 0.008 - ETA: 6:20 - loss: 0.008 - ETA: 6:19 - loss: 0.008 - ETA: 6:18 - loss: 0.008 - ETA: 6:17 - loss: 0.008 - ETA: 6:17 - loss: 0.008 - ETA: 6:16 - loss: 0.008 - ETA: 6:15 - loss: 0.008 - ETA: 6:14 - loss: 0.008 - ETA: 6:13 - loss: 0.008 - ETA: 6:13 - loss: 0.008 - ETA: 6:12 - loss: 0.008 - ETA: 6:11 - los

100000/100000 [==============================] - ETA: 2:31 - loss: 0.008 - ETA: 2:31 - loss: 0.008 - ETA: 2:30 - loss: 0.008 - ETA: 2:29 - loss: 0.008 - ETA: 2:28 - loss: 0.008 - ETA: 2:27 - loss: 0.008 - ETA: 2:27 - loss: 0.008 - ETA: 2:26 - loss: 0.008 - ETA: 2:25 - loss: 0.008 - ETA: 2:24 - loss: 0.008 - ETA: 2:23 - loss: 0.008 - ETA: 2:23 - loss: 0.008 - ETA: 2:22 - loss: 0.008 - ETA: 2:21 - loss: 0.008 - ETA: 2:20 - loss: 0.008 - ETA: 2:19 - loss: 0.008 - ETA: 2:19 - loss: 0.008 - ETA: 2:18 - loss: 0.008 - ETA: 2:17 - loss: 0.008 - ETA: 2:16 - loss: 0.008 - ETA: 2:16 - loss: 0.008 - ETA: 2:15 - loss: 0.008 - ETA: 2:14 - loss: 0.008 - ETA: 2:13 - loss: 0.008 - ETA: 2:12 - loss: 0.008 - ETA: 2:12 - loss: 0.008 - ETA: 2:11 - loss: 0.008 - ETA: 2:10 - loss: 0.008 - ETA: 2:09 - loss: 0.008 - ETA: 2:08 - loss: 0.008 - ETA: 2:08 - loss: 0.008 - ETA: 2:07 - loss: 0.008 - ETA: 2:06 - loss: 0.008 - ETA: 2:05 - loss: 0.008 - ETA: 2:04 - loss: 0.008 - ETA: 2:04 - loss: 0.008 - ETA: 2:03 - los

In [19]:
custom_hist = CustomHistory()
custom_hist.init()

early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

model.fit(x_data, y_data, epochs=1000, batch_size=1, shuffle=False, callbacks=[early_stop])
model.reset_states()

Epoch 1/1000


   272/100000 [..............................] - ETA: 151:54:24 - loss: 0.00 - ETA: 115:00:39 - loss: 0.00 - ETA: 95:36:36 - loss: 0.0082 - ETA: 84:16:22 - loss: 0.008 - ETA: 77:44:12 - loss: 0.009 - ETA: 73:54:55 - loss: 0.009 - ETA: 71:14:56 - loss: 0.008 - ETA: 68:39:10 - loss: 0.008 - ETA: 66:32:49 - loss: 0.009 - ETA: 65:12:23 - loss: 0.009 - ETA: 64:05:21 - loss: 0.008 - ETA: 63:17:15 - loss: 0.009 - ETA: 62:14:01 - loss: 0.009 - ETA: 61:36:27 - loss: 0.009 - ETA: 60:54:34 - loss: 0.009 - ETA: 60:13:20 - loss: 0.009 - ETA: 59:44:01 - loss: 0.009 - ETA: 59:16:50 - loss: 0.009 - ETA: 58:49:00 - loss: 0.009 - ETA: 58:28:16 - loss: 0.009 - ETA: 57:59:23 - loss: 0.009 - ETA: 57:54:17 - loss: 0.009 - ETA: 57:43:51 - loss: 0.009 - ETA: 57:36:13 - loss: 0.009 - ETA: 57:24:24 - loss: 0.009 - ETA: 57:05:49 - loss: 0.009 - ETA: 56:52:03 - loss: 0.009 - ETA: 56:42:07 - loss: 0.009 - ETA: 56:36:46 - loss: 0.009 - ETA: 56:32:13 - loss: 0.009 - ETA: 56:18:44 - loss: 0.009 - ETA: 56:13:34 - loss

  2176/100000 [..............................] - ETA: 57:48:38 - loss: 0.008 - ETA: 57:48:40 - loss: 0.008 - ETA: 57:48:42 - loss: 0.008 - ETA: 57:48:46 - loss: 0.008 - ETA: 57:48:48 - loss: 0.008 - ETA: 57:48:53 - loss: 0.008 - ETA: 57:48:56 - loss: 0.008 - ETA: 57:49:07 - loss: 0.008 - ETA: 57:49:09 - loss: 0.008 - ETA: 57:49:16 - loss: 0.008 - ETA: 57:49:19 - loss: 0.008 - ETA: 57:49:22 - loss: 0.008 - ETA: 57:49:25 - loss: 0.008 - ETA: 57:49:28 - loss: 0.008 - ETA: 57:49:31 - loss: 0.008 - ETA: 57:49:36 - loss: 0.008 - ETA: 57:49:40 - loss: 0.008 - ETA: 57:49:41 - loss: 0.008 - ETA: 57:49:47 - loss: 0.008 - ETA: 57:49:48 - loss: 0.008 - ETA: 57:49:52 - loss: 0.008 - ETA: 57:49:56 - loss: 0.008 - ETA: 57:50:00 - loss: 0.008 - ETA: 57:50:05 - loss: 0.008 - ETA: 57:50:04 - loss: 0.008 - ETA: 57:50:08 - loss: 0.008 - ETA: 57:50:11 - loss: 0.008 - ETA: 57:50:14 - loss: 0.008 - ETA: 57:50:16 - loss: 0.008 - ETA: 57:50:17 - loss: 0.008 - ETA: 57:50:19 - loss: 0.008 - ETA: 57:50:24 - loss:

  2448/100000 [..............................] - ETA: 57:59:09 - loss: 0.008 - ETA: 57:59:11 - loss: 0.008 - ETA: 57:59:10 - loss: 0.008 - ETA: 57:59:10 - loss: 0.008 - ETA: 57:59:11 - loss: 0.008 - ETA: 57:59:11 - loss: 0.008 - ETA: 57:59:11 - loss: 0.008 - ETA: 57:59:17 - loss: 0.008 - ETA: 57:59:17 - loss: 0.008 - ETA: 57:59:19 - loss: 0.008 - ETA: 57:59:29 - loss: 0.008 - ETA: 57:59:33 - loss: 0.008 - ETA: 57:59:36 - loss: 0.008 - ETA: 57:59:34 - loss: 0.008 - ETA: 57:59:35 - loss: 0.008 - ETA: 57:59:37 - loss: 0.008 - ETA: 57:59:38 - loss: 0.008 - ETA: 57:59:40 - loss: 0.008 - ETA: 57:59:40 - loss: 0.008 - ETA: 57:59:42 - loss: 0.008 - ETA: 57:59:45 - loss: 0.008 - ETA: 57:59:45 - loss: 0.008 - ETA: 57:59:43 - loss: 0.008 - ETA: 57:59:47 - loss: 0.008 - ETA: 57:59:49 - loss: 0.008 - ETA: 57:59:48 - loss: 0.008 - ETA: 57:59:49 - loss: 0.008 - ETA: 57:59:50 - loss: 0.008 - ETA: 57:59:48 - loss: 0.008 - ETA: 57:59:49 - loss: 0.008 - ETA: 57:59:52 - loss: 0.008 - ETA: 57:59:51 - loss:

  2720/100000 [..............................] - ETA: 57:30:54 - loss: 0.008 - ETA: 57:30:44 - loss: 0.008 - ETA: 57:30:35 - loss: 0.008 - ETA: 57:30:23 - loss: 0.008 - ETA: 57:30:17 - loss: 0.008 - ETA: 57:30:10 - loss: 0.008 - ETA: 57:30:00 - loss: 0.008 - ETA: 57:29:51 - loss: 0.008 - ETA: 57:29:43 - loss: 0.008 - ETA: 57:29:35 - loss: 0.008 - ETA: 57:29:27 - loss: 0.008 - ETA: 57:29:16 - loss: 0.008 - ETA: 57:29:08 - loss: 0.008 - ETA: 57:28:59 - loss: 0.008 - ETA: 57:28:50 - loss: 0.008 - ETA: 57:28:39 - loss: 0.008 - ETA: 57:28:28 - loss: 0.008 - ETA: 57:28:18 - loss: 0.008 - ETA: 57:28:09 - loss: 0.008 - ETA: 57:27:59 - loss: 0.008 - ETA: 57:27:49 - loss: 0.008 - ETA: 57:27:39 - loss: 0.008 - ETA: 57:27:32 - loss: 0.008 - ETA: 57:27:35 - loss: 0.008 - ETA: 57:27:26 - loss: 0.008 - ETA: 57:27:15 - loss: 0.008 - ETA: 57:27:07 - loss: 0.008 - ETA: 57:26:59 - loss: 0.008 - ETA: 57:26:50 - loss: 0.008 - ETA: 57:26:39 - loss: 0.008 - ETA: 57:26:31 - loss: 0.008 - ETA: 57:26:19 - loss:

...

  5712/100000 [>.............................] - ETA: 53:13:41 - loss: 0.008 - ETA: 53:13:37 - loss: 0.008 - ETA: 53:13:37 - loss: 0.008 - ETA: 53:13:33 - loss: 0.008 - ETA: 53:13:29 - loss: 0.008 - ETA: 53:13:24 - loss: 0.008 - ETA: 53:13:22 - loss: 0.008 - ETA: 53:13:19 - loss: 0.008 - ETA: 53:13:15 - loss: 0.008 - ETA: 53:13:11 - loss: 0.008 - ETA: 53:13:07 - loss: 0.008 - ETA: 53:13:04 - loss: 0.008 - ETA: 53:13:01 - loss: 0.008 - ETA: 53:12:57 - loss: 0.008 - ETA: 53:12:53 - loss: 0.008 - ETA: 53:12:50 - loss: 0.008 - ETA: 53:12:46 - loss: 0.008 - ETA: 53:12:42 - loss: 0.008 - ETA: 53:12:39 - loss: 0.008 - ETA: 53:12:36 - loss: 0.008 - ETA: 53:12:32 - loss: 0.008 - ETA: 53:12:29 - loss: 0.008 - ETA: 53:12:25 - loss: 0.008 - ETA: 53:12:22 - loss: 0.008 - ETA: 53:12:19 - loss: 0.008 - ETA: 53:12:15 - loss: 0.008 - ETA: 53:12:12 - loss: 0.008 - ETA: 53:12:08 - loss: 0.008 - ETA: 53:12:06 - loss: 0.008 - ETA: 53:12:02 - loss: 0.008 - ETA: 53:11:58 - loss: 0.008 - ETA: 53:11:56 - loss:

  5984/100000 [>.............................] - ETA: 53:01:16 - loss: 0.008 - ETA: 53:01:14 - loss: 0.008 - ETA: 53:01:11 - loss: 0.008 - ETA: 53:01:07 - loss: 0.008 - ETA: 53:01:03 - loss: 0.008 - ETA: 53:01:00 - loss: 0.008 - ETA: 53:00:56 - loss: 0.008 - ETA: 53:00:52 - loss: 0.008 - ETA: 53:00:49 - loss: 0.008 - ETA: 53:00:46 - loss: 0.008 - ETA: 53:00:42 - loss: 0.008 - ETA: 53:00:37 - loss: 0.008 - ETA: 53:00:34 - loss: 0.008 - ETA: 53:00:32 - loss: 0.008 - ETA: 53:00:28 - loss: 0.008 - ETA: 53:00:24 - loss: 0.008 - ETA: 53:00:21 - loss: 0.008 - ETA: 53:00:18 - loss: 0.008 - ETA: 53:00:15 - loss: 0.008 - ETA: 53:00:11 - loss: 0.008 - ETA: 53:00:07 - loss: 0.008 - ETA: 53:00:03 - loss: 0.008 - ETA: 53:00:01 - loss: 0.008 - ETA: 52:59:57 - loss: 0.008 - ETA: 52:59:53 - loss: 0.008 - ETA: 52:59:49 - loss: 0.008 - ETA: 52:59:46 - loss: 0.008 - ETA: 52:59:43 - loss: 0.008 - ETA: 52:59:40 - loss: 0.008 - ETA: 52:59:36 - loss: 0.008 - ETA: 52:59:32 - loss: 0.008 - ETA: 52:59:28 - loss:

  6256/100000 [>.............................] - ETA: 52:47:41 - loss: 0.008 - ETA: 52:47:37 - loss: 0.008 - ETA: 52:47:34 - loss: 0.008 - ETA: 52:47:30 - loss: 0.008 - ETA: 52:47:28 - loss: 0.008 - ETA: 52:47:24 - loss: 0.008 - ETA: 52:47:21 - loss: 0.008 - ETA: 52:47:18 - loss: 0.008 - ETA: 52:47:15 - loss: 0.008 - ETA: 52:47:12 - loss: 0.008 - ETA: 52:47:10 - loss: 0.008 - ETA: 52:47:06 - loss: 0.008 - ETA: 52:47:02 - loss: 0.008 - ETA: 52:46:59 - loss: 0.008 - ETA: 52:46:57 - loss: 0.008 - ETA: 52:46:54 - loss: 0.008 - ETA: 52:46:51 - loss: 0.008 - ETA: 52:46:47 - loss: 0.008 - ETA: 52:46:44 - loss: 0.008 - ETA: 52:46:42 - loss: 0.008 - ETA: 52:46:38 - loss: 0.008 - ETA: 52:46:34 - loss: 0.008 - ETA: 52:46:31 - loss: 0.008 - ETA: 52:46:27 - loss: 0.008 - ETA: 52:46:25 - loss: 0.008 - ETA: 52:46:21 - loss: 0.008 - ETA: 52:46:18 - loss: 0.008 - ETA: 52:46:15 - loss: 0.008 - ETA: 52:46:10 - loss: 0.008 - ETA: 52:46:07 - loss: 0.008 - ETA: 52:46:04 - loss: 0.008 - ETA: 52:46:01 - loss:

  6496/100000 [>.............................] - ETA: 52:33:31 - loss: 0.008 - ETA: 52:33:28 - loss: 0.008 - ETA: 52:33:24 - loss: 0.008 - ETA: 52:33:22 - loss: 0.008 - ETA: 52:33:19 - loss: 0.008 - ETA: 52:33:15 - loss: 0.008 - ETA: 52:33:12 - loss: 0.008 - ETA: 52:33:09 - loss: 0.008 - ETA: 52:33:06 - loss: 0.008 - ETA: 52:33:04 - loss: 0.008 - ETA: 52:33:00 - loss: 0.008 - ETA: 52:32:57 - loss: 0.008 - ETA: 52:32:54 - loss: 0.008 - ETA: 52:32:52 - loss: 0.008 - ETA: 52:32:48 - loss: 0.008 - ETA: 52:32:44 - loss: 0.008 - ETA: 52:32:41 - loss: 0.008 - ETA: 52:32:38 - loss: 0.008 - ETA: 52:32:36 - loss: 0.008 - ETA: 52:32:32 - loss: 0.008 - ETA: 52:32:29 - loss: 0.008 - ETA: 52:32:26 - loss: 0.008 - ETA: 52:32:22 - loss: 0.008 - ETA: 52:32:19 - loss: 0.008 - ETA: 52:32:16 - loss: 0.008 - ETA: 52:32:13 - loss: 0.008 - ETA: 52:32:09 - loss: 0.008 - ETA: 52:32:06 - loss: 0.008 - ETA: 52:32:03 - loss: 0.008 - ETA: 52:31:59 - loss: 0.008 - ETA: 52:31:56 - loss: 0.008 - ETA: 52:31:52 - loss:

KeyboardInterrupt: 

In [20]:
# 검증 wav 파일로부터 Feature를 만듭니다.
x_test = glob('test/*.wav')
x_test = data_loader(x_test)
x_test = x_test / 30000
x_test = x_test[:, ::8]
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)


 19%|██████████████▋                                                             | 1927/10000 [00:14<00:58, 137.61it/s]


 39%|█████████████████████████████▊                                              | 3918/10000 [00:29<00:42, 142.20it/s]


 60%|█████████████████████████████████████████████▎                              | 5962/10000 [00:43<00:30, 133.58it/s]


 80%|████████████████████████████████████████████████████████████▊               | 7998/10000 [00:58<00:14, 135.14it/s]


 99%|███████████████████████████████████████████████████████████████████████████▍| 9918/10000 [01:13<00:00, 137.26it/s]


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [01:14<00:00, 135.10it/s]


In [23]:
x_test.shape

(10000, 2000, 1)

In [37]:
y_pred = model.predict(x_test)

In [38]:
y_pred = model.predict(x_test)
# 예측 결과로 제출 파일을 생성합니다.
submission = pd.read_csv('submission.csv', index_col=0)
submission.loc[:, :] = y_pred
submission.to_csv('submission.csv')